<a href="https://colab.research.google.com/github/SlyFox579/bdt-2023-25962701/blob/main/pyspark_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An implementation for porting to other platforms and discussion (this is not to do exploratory analysis but rather to consider the APIs and technologies involved - it is not intended to be a good or reference solution to this problem).

Obtain the data from Google Cloud Storage buckets

In [123]:
! wget https://storage.googleapis.com/bdt-spark-store/external_sources.csv -O gcs_external_sources.csv

--2023-11-06 19:04:48--  https://storage.googleapis.com/bdt-spark-store/external_sources.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c0d::cf, 2607:f8b0:4023:c03::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15503836 (15M) [text/csv]
Saving to: ‘gcs_external_sources.csv’

gcs_external_source 100%[===================>]  14.79M  10.6MB/s    in 1.4s    

2023-11-06 19:04:50 (10.6 MB/s) - ‘gcs_external_sources.csv’ saved [15503836/15503836]



In [124]:
! wget https://storage.googleapis.com/bdt-spark-store/internal_data.csv -O gcs_internal_data.csv

--2023-11-06 19:04:53--  https://storage.googleapis.com/bdt-spark-store/internal_data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c0d::cf, 2607:f8b0:4023:c03::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152978396 (146M) [text/csv]
Saving to: ‘gcs_internal_data.csv’

gcs_internal_data.c 100%[===================>] 145.89M  25.1MB/s    in 6.5s    

2023-11-06 19:05:01 (22.3 MB/s) - ‘gcs_internal_data.csv’ saved [152978396/152978396]



In [125]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


In [126]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,455 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,279 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,013 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,186 kB]
Hit:13 https://ppa.launchpadcontent.n

In [127]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [128]:
# get spark
VERSION='3.5.0'
!wget https://dlcdn.apache.org/spark/spark-$VERSION/spark-$VERSION-bin-hadoop3.tgz

--2023-11-06 19:05:24--  https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400395283 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.0-bin-hadoop3.tgz.1’

spark-3.5.0-bin-had 100%[===================>] 381.85M   198MB/s    in 1.9s    

2023-11-06 19:05:26 (198 MB/s) - ‘spark-3.5.0-bin-hadoop3.tgz.1’ saved [400395283/400395283]



In [129]:
# decompress spark
!tar xf spark-$VERSION-bin-hadoop3.tgz

# install python package to help with system paths
!pip install -q findspark

In [130]:
# Let Colab know where the java and spark folders are

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{VERSION}-bin-hadoop3"

In [131]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("local[*]").getOrCreate()

Read in data sources

In [132]:
# Read a CSV file and create a DataFrame
df_data = spark.read.csv("gcs_internal_data.csv", header=True, inferSchema=True)
df_ext = spark.read.csv("gcs_external_sources.csv", header=True, inferSchema=True)

Join them on their common identifier key

In [133]:
# Perform an inner join on the 'SK_ID_CURR' column
df_full = df_data.join(df_ext, on='SK_ID_CURR', how='inner')

We will filter a few features out for the sake of this example

In [134]:
# List of columns to extract
columns_extract = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                  'DAYS_BIRTH', 'DAYS_EMPLOYED', 'NAME_EDUCATION_TYPE',
                  'DAYS_ID_PUBLISH', 'CODE_GENDER', 'AMT_ANNUITY',
                  'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
                  'ORGANIZATION_TYPE', 'DAYS_LAST_PHONE_CHANGE',
                  'NAME_INCOME_TYPE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE', 'TARGET']

# Select the specified columns from 'df_full'
df = df_full.select(columns_extract)

Let's obtain a train and test split

In [135]:
import random

# Set a specific seed value (e.g., 101)
seed_value = 101

# Create a random number generator with the specified seed
rand_gen = random.Random(seed_value)

# Use the rand_gen to generate random numbers


In [136]:
from pyspark.sql.functions import rand

# Randomly shuffle the DataFrame
df = df.orderBy(rand())

# Split the DataFrame into training and test sets
split_ratio = 0.8  # 80% for training, 20% for testing
split_seed = 42     # Set a specific seed for reproducibility

train, test = df.randomSplit([split_ratio, 1 - split_ratio], seed=split_seed)


In [137]:
from pyspark.sql.functions import col, count

# Calculate value counts and relative frequencies for 'TARGET' column in train
train_value_counts = train.groupBy('TARGET').agg(count('*').alias('count'))
train_relative_frequencies = train_value_counts.withColumn(
    'Relative_Frequency', col('count') / train.count()
)

# Calculate value counts and relative frequencies for 'TARGET' column in test
test_value_counts = test.groupBy('TARGET').agg(count('*').alias('count'))
test_relative_frequencies = test_value_counts.withColumn(
    'Relative_Frequency', col('count') / test.count()
)

# Show the results
print("Train Data:")
train_relative_frequencies.show()

print("Test Data:")
test_relative_frequencies.show()


Train Data:
+------+------+-------------------+
|TARGET| count| Relative_Frequency|
+------+------+-------------------+
|     1| 19905|0.08092351599565806|
|     0|226055|  0.919023632675131|
+------+------+-------------------+

Test Data:
+------+-----+-------------------+
|TARGET|count| Relative_Frequency|
+------+-----+-------------------+
|     1| 4887|0.07940531318547404|
|     0|56661| 0.9206434316353888|
+------+-----+-------------------+



Handle the categorical variables

In [138]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# List of categorical column names to one-hot encode
categorical_columns = ['NAME_EDUCATION_TYPE', 'CODE_GENDER', 'ORGANIZATION_TYPE', 'NAME_INCOME_TYPE']  # Replace with your column names

# Create an empty list to store one-hot encoded column names
one_hot_encoded_cols = []

# Create a Pipeline for one-hot encoding each categorical column
for col_name in categorical_columns:
    # Create a StringIndexer for the current column
    indexer = StringIndexer(inputCol=col_name, outputCol=col_name + '_index')

    # Create a OneHotEncoder for the indexed column
    encoder = OneHotEncoder(inputCol=col_name + '_index', outputCol=col_name + '_encoded')

    # Append the one-hot encoded column name to the list
    one_hot_encoded_cols.append(col_name + '_encoded')

    # Define the stages of the pipeline
    stages = [indexer, encoder]

    # Create the pipeline
    pipeline = Pipeline(stages=stages)

    # Fit and transform the train and test DataFrames
    train = pipeline.fit(train).transform(train)
    test = pipeline.fit(test).transform(test)

# Print the shapes of the resulting DataFrames
print('Training Features shape:', (train.count(), len(train.columns)))
print('Testing Features shape:', (test.count(), len(test.columns)))


Training Features shape: (245961, 26)
Testing Features shape: (61554, 26)


Align the training and test data (as the test data may not have the same columns in the encoding)

In [139]:
from pyspark.sql.functions import col

# List of common column names between train and test DataFrames
common_columns = [col_name for col_name in train.columns if col_name in test.columns]

# Select only the common columns in both train and test DataFrames
train = train.select(*common_columns)
test = test.select(*common_columns)

# Print the shapes of the resulting DataFrames
print('Training Features shape:', (train.count(), len(train.columns)))
print('Testing Features shape:', (test.count(), len(test.columns)))


Training Features shape: (245967, 26)
Testing Features shape: (61545, 26)


In [144]:
# List of columns to extract
columns_extract = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                  'DAYS_BIRTH', 'DAYS_EMPLOYED', 'NAME_EDUCATION_TYPE_index',
                  'DAYS_ID_PUBLISH', 'CODE_GENDER_index', 'AMT_ANNUITY',
                  'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
                  'ORGANIZATION_TYPE_index', 'DAYS_LAST_PHONE_CHANGE',
                  'NAME_INCOME_TYPE_index', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE', 'TARGET']

# Select the specified columns from 'df_full'
train = train.select(columns_extract)
test = test.select(columns_extract)

# Print the shapes of the resulting DataFrames
print('Training Features shape:', (train.count(), len(train.columns)))
print('Testing Features shape:', (test.count(), len(test.columns)))


Training Features shape: (245970, 18)
Testing Features shape: (61549, 18)


Fill in missing data and scale

In [145]:
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import functions as F

# Feature names
features = train.columns

# Median imputation of missing values
imputer = Imputer(inputCols=features, outputCols=features, strategy='median')
imputer_model = imputer.fit(train)

train = imputer_model.transform(train)
test = imputer_model.transform(test)

# Create a feature vector assembler
vector_assembler = VectorAssembler(inputCols=features, outputCol="features")
train = vector_assembler.transform(train)
test = vector_assembler.transform(test)

# Scale each feature to 0-1
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(train)

train = scaler_model.transform(train)
test = scaler_model.transform(test)

# Print data shapes
print('Training data shape:', (train.count(), len(train.columns)))
print('Testing data shape:', (test.count(), len(test.columns)))


Training data shape: (245968, 20)
Testing data shape: (61547, 20)


Fit random forest

In [146]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create a Random Forest Classifier
random_forest = RandomForestClassifier(
    labelCol='TARGET',
    featuresCol='scaled_features',  # Replace with your feature column name
    numTrees=100,
    featureSubsetStrategy="auto",
    seed=50,
    subsamplingRate=1.0,
    maxDepth=5,
    impurity="gini",
    minInstancesPerNode=1,
    minInfoGain=0.0,
    maxMemoryInMB=256,
    cacheNodeIds=False,
    checkpointInterval=10,
    maxBins=32,
    minWeightFractionPerNode=0.0,
)

# Create a pipeline for data preparation and training
pipeline = Pipeline(stages=[random_forest])

# Train the model on the training data
model = pipeline.fit(train)

# Make predictions on the test data
predictions = model.transform(test)

# Extract feature importances
feature_importances = model.stages[-1].featureImportances

# Evaluate the model (You may need to adjust this depending on your evaluation metric)
evaluator = BinaryClassificationEvaluator(labelCol="TARGET", rawPredictionCol="prediction")
area_under_curve = evaluator.evaluate(predictions)

print("Area Under ROC:", area_under_curve)


Area Under ROC: 1.0


In [147]:
feature_importances

SparseVector(18, {0: 0.0035, 1: 0.0101, 2: 0.012, 3: 0.0015, 4: 0.0003, 5: 0.0003, 6: 0.0, 7: 0.0004, 8: 0.0, 9: 0.0, 10: 0.0002, 11: 0.0, 13: 0.0004, 14: 0.0003, 15: 0.0, 16: 0.0, 17: 0.971})